# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get the PRICE_DATA environment variable
price_data_path = os.getenv('PRICE_DATA')

if price_data_path:
    # Load the parquet file into a pandas DataFrame
    df = pd.read_parquet(price_data_path)
    
    # Display the first few rows of the DataFrame
    print(df.head())
else:
    print("PRICE_DATA environment variable not set.")


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.613007   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.281456   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.782787   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.344173   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.372852   

         Volume       sector                       subsector  year  
ticker                                                              
A       4674353  Health Care  Life Sciences Tools & Services  2000  
A       4765083  Health Care  Life Sciences Tools & Services  2000  
A       5758642  Health Care  Life Sciences Tools & Services  2000  
A       2534434  Health Care  Life Sciences Tools & Services  2000  
A       2819626  Health Care  Life Sciences Tools & S

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Import necessary libraries
import os
from dotenv import load_dotenv
import glob

# Specify the path to your .env file
dotenv_path = 'c:/Users/4372/scaling_to_production/01_materials/.env'

# Load environment variables from the specified .env file
load_dotenv(dotenv_path=dotenv_path)

# Get the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')

# Check if the PRICE_DATA variable is set
if price_data_dir:
    # Use glob to find all parquet files in the directory
    parquet_files = glob.glob(os.path.join(price_data_dir, '*.parquet'))

    # Display the list of parquet files found
    print("Parquet files found in PRICE_DATA directory:")
    for file in parquet_files:
        print(file)
else:
    print("PRICE_DATA environment variable not set.")




Parquet files found in PRICE_DATA directory:


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [35]:
import dask.dataframe as dd

data_path = r'C:\Users\14372\scaling_to_production\05_src\data\temp\stock_px.csv'
ddf = dd.read_csv(data_path)

ddf['Close'] = ddf.groupby('ticker')['Close'].shift(1)

ddf['returns'] = (ddf['Close'] / ddf['Close']) - 1

ddf['hi_lo_range'] = ddf['High'] - ddf['Low']
dd_feat = ddf[['ticker', 'Date', 'Close', 'returns', 'hi_lo_range']]

C:\Users\14372\AppData\Local\Temp\ipykernel_14568\3411697288.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Close'] = ddf.groupby('ticker')['Close'].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [36]:
import dask.dataframe as dd
import pandas as pd

# Example Dask DataFrame
data = {'returns': [0.01, 0.02, -0.01, 0.03, 0.04, -0.02, 0.01, 0.05, 0.03, 0.02, 0.01, -0.01]}
ddf = dd.from_pandas(pd.DataFrame(data), npartitions=2)

# Convert to pandas DataFrame
pdf = ddf.compute()

# Add rolling average return calculation
pdf['rolling_avg_return'] = pdf['returns'].rolling(window=10).mean()

print(pdf)

    returns  rolling_avg_return
0      0.01                 NaN
1      0.02                 NaN
2     -0.01                 NaN
3      0.03                 NaN
4      0.04                 NaN
5     -0.02                 NaN
6      0.01                 NaN
7      0.05                 NaN
8      0.03                 NaN
9      0.02               0.018
10     0.01               0.018
11    -0.01               0.015


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not at all necessary to convert to Pandas to calculate the moving average return. Dask DataFrames are designed to handle large datasets that do not fit into memory and provide a similar interface to Pandas. Operations such as calculating a moving average can be performed directly on a Dask DataFrame without needing to convert it to a Pandas DataFrame.

Yes, it would have been better to perform the calculation in Dask if you are working with large datasets. Here are the reasons:

1. Scalability: Dask can handle larger-than-memory datasets by parallelizing the computation across multiple cores or even a distributed cluster. This makes it suitable for large datasets where Pandas would be limited by available memory.
2. Performance: By leveraging parallelism, Dask can potentially speed up computations compared to Pandas, which operates on a single core.
3. Memory Efficiency: Dask processes data in chunks, which allows it to manage memory usage more efficiently than Pandas.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.